# Khipus.ai
## Neural Network from Scratch
### Case Study: 
<span>© Copyright Notice 2025, Khipus.ai - All Rights Reserved.</span>
---
This notebook implements a simple neural network from scratch using NumPy. We will build, train, and evaluate a basic single-layer perceptron on a heart disease dataset.

## 1. Importing Required Packages
We begin by importing necessary Python libraries for data processing, visualization, and model building.

In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


## 2. Loading and Preprocessing the Dataset
We use the heart disease dataset to train our neural network.

In [ ]:

# Load dataset (Assume 'heart.csv' is available)
df = pd.read_csv('heart.csv')

# Display the first few rows of the dataset
df.head()


## 3. Selecting Features and Labels
We separate the independent variables (features) from the dependent variable (target).

In [ ]:

# Extract features and labels
X = np.array(df.loc[:, df.columns != 'output'])
y = np.array(df['output'])

# Print dataset shapes
print(f"X: {X.shape}, y: {y.shape}")


## 4. Splitting Data into Training and Testing Sets
We split the dataset into 80% training and 20% testing sets.

In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


## 5. Scaling Features
To improve training performance, we scale the feature values.

In [ ]:

# Standardizing the features
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)


## 6. Implementing a Neural Network from Scratch
We define a single-layer perceptron with a sigmoid activation function.

In [ ]:

# Define the Neural Network class
class NeuralNetworkFromScratch:
    def __init__(self, LR, X_train, y_train, X_test, y_test):
        self.w = np.random.randn(X_train.shape[1])
        self.b = np.random.randn()
        self.LR = LR
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.L_train = []
        self.L_test = []
        
    def activation(self, x):
        # Sigmoid function
        return 1 / (1 + np.exp(-x))

    def dactivation(self, x):
        # Derivative of sigmoid function
        return self.activation(x) * (1 - self.activation(x))

    def forward(self, X):
        hidden_1 = np.dot(X, self.w) + self.b
        activate_1 = self.activation(hidden_1)
        return activate_1

    def backward(self, X, y_true):
        # Compute gradients
        hidden_1 = np.dot(X, self.w) + self.b
        y_pred = self.forward(X)
        dL_dpred = 2 * (y_pred - y_true)
        dpred_dhidden1 = self.dactivation(hidden_1)
        dhidden1_db = 1
        dhidden1_dw = X

        dL_db = dL_dpred * dpred_dhidden1 * dhidden1_db
        dL_dw = dL_dpred * dpred_dhidden1 * dhidden1_dw
        return dL_db, dL_dw

    def optimizer(self, dL_db, dL_dw):
        # Update weights
        self.b = self.b - dL_db * self.LR
        self.w = self.w - dL_dw * self.LR

    def train(self, ITERATIONS):
        for i in range(ITERATIONS):
            random_pos = np.random.randint(len(self.X_train))
            y_train_true = self.y_train[random_pos]
            y_train_pred = self.forward(self.X_train[random_pos])
            
            # Compute training loss
            L = np.sum(np.square(y_train_pred - y_train_true))
            self.L_train.append(L)
            
            # Compute gradients
            dL_db, dL_dw = self.backward(
                self.X_train[random_pos], self.y_train[random_pos]
            )
            
            # Update weights
            self.optimizer(dL_db, dL_dw)

            # Compute test error
            L_sum = 0
            for j in range(len(self.X_test)):
                y_true = self.y_test[j]
                y_pred = self.forward(self.X_test[j])
                L_sum += np.square(y_pred - y_true)
            self.L_test.append(L_sum)

        return "Training successfully finished"


## 7. Training the Model
We train the neural network using a simple gradient descent approach.

In [ ]:

# Hyperparameters
LR = 0.1
ITERATIONS = 1000

# Create and train the model
nn = NeuralNetworkFromScratch(LR=LR, X_train=X_train_scale, y_train=y_train, X_test=X_test_scale, y_test=y_test)
nn.train(ITERATIONS=ITERATIONS)


## 8. Visualizing Training Loss
We plot the loss over training iterations to observe convergence.

In [ ]:

# Plot loss
sns.lineplot(x=list(range(len(nn.L_test))), y=nn.L_test)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Loss over Training Iterations")
plt.show()


## 9. Evaluating the Model
We test the model's accuracy by making predictions on the test dataset.

In [ ]:

# Model evaluation
total = X_test_scale.shape[0]
correct = 0
y_preds = []

for i in range(total):
    y_true = y_test[i]
    y_pred = np.round(nn.forward(X_test_scale[i]))
    y_preds.append(y_pred)
    correct += 1 if y_true == y_pred else 0

# Calculate accuracy
accuracy = correct / total
print(f"Model Accuracy: {accuracy * 100:.2f}%")


## 10. Confusion Matrix
We compute a confusion matrix to analyze classification performance.

In [ ]:

# Compute confusion matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_preds)
print("Confusion Matrix:")
print(cm)
